### 輸入資料

In [32]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
# from google.colab import drive

# drive.mount('/content/drive')
print("hello world")
sideORbottom = "side" #side/bottom
version = "all"

hello world


In [33]:
import re
mypath = 'K:\\DataMiningProject\\Training_set\\'

x = []
y = []

count = 0
for (dirpath, dirnames, filenames) in os.walk(mypath):
  dirpath_list = dirpath.split('\\')
  # if count > 20:
  #   break


  if len(dirpath_list) > 5:
    for f in filenames:
      if ".jpg" in f:

        # if 'pine-bottom' in dirpath:
        # if 'pine-side' in dirpath:
        if f'pine-{sideORbottom}' in dirpath:
          count += 1
          # if count > 20:
          #   break

          print(f, dirnames, dirpath_list[3],{dirpath_list[4]}, dirpath_list[5])
          image_path = f'{dirpath}/{f}'
          image = keras.utils.load_img(image_path, color_mode="grayscale", target_size=(360, 640))
          input_arr = np.array([keras.utils.img_to_array(image)])  # Convert single image to a batch.
          x.append(input_arr)
          y.append(f"{dirpath_list[3]}_{dirpath_list[4]}_{dirpath_list[5]}_{f}")


01.jpg [] 0010 {'loc-1'} pine-side
02.jpg [] 0010 {'loc-1'} pine-side
02.jpg [] 0011 {'loc-2'} pine-side
04.jpg [] 0011 {'loc-2'} pine-side
03.jpg [] 0014 {'loc-1'} pine-side
02.jpg [] 0019 {'loc-2'} pine-side
01.jpg [] 0021 {'loc-2'} pine-side
02.jpg [] 0023 {'loc-2'} pine-side
02.jpg [] 0029 {'loc-1'} pine-side
01.jpg [] 0040 {'loc-2'} pine-side
02.jpg [] 0045 {'loc-2'} pine-side
02.jpg [] 0052 {'loc-2'} pine-side
01.jpg [] 0065 {'loc-1'} pine-side
01.jpg [] 0096 {'loc-2'} pine-side
02.jpg [] 0096 {'loc-2'} pine-side
03.jpg [] 0096 {'loc-2'} pine-side
03.jpg [] 0097 {'loc-2'} pine-side
01.jpg [] 0101 {'loc-2'} pine-side
04.jpg [] 0101 {'loc-2'} pine-side
03.jpg [] 0102 {'loc-2'} pine-side
03.jpg [] 0127 {'loc-2'} pine-side
03.jpg [] 0162 {'loc-2'} pine-side
04.jpg [] 0162 {'loc-2'} pine-side
02.jpg [] 0187 {'loc-2'} pine-side
03.jpg [] 0187 {'loc-2'} pine-side
01.jpg [] 0188 {'loc-1'} pine-side
01.jpg [] 0209 {'loc-2'} pine-side
02.jpg [] 0209 {'loc-2'} pine-side
03.jpg [] 0209 {'loc

### 訓練資料整合

#### train_label

In [34]:

train_label = pd.read_csv(f'{mypath}/training_set_label.csv')
print(train_label)
train_label.rename(columns={'1': 'ID', '1.1': 'Classification'}, inplace=True)

# 要插入的行的數據
new_row_data = {'ID': 1, 'Classification': 1}
new_row_df = pd.DataFrame([new_row_data])

# concat 函數將新的行插入到 DataFrame 的最開頭
train_label = pd.concat([new_row_df, train_label], ignore_index=True)

train_label.to_csv('train_label.csv', index=False)
train_label

       1  1.1
0      2    1
1      3    1
2      4    1
3      6    1
4      7    1
..   ...  ...
394  495    4
395  496    3
396  497    4
397  498    4
398  500    4

[399 rows x 2 columns]


,ID,Classification
0,1,1
1,2,1
2,3,1
3,4,1
4,6,1
...,...,...
395,495,4
396,496,3
397,497,4
398,498,4


#### train_y

In [35]:
# 轉換 y 為 DataFrame
y_df = pd.DataFrame({'Folder': y})
y_df
# 函數，提取 Folder 的前四個字符
def extract_prefix(folder):
    return int(folder[:4])

# ID = Folder 的前四個字符
y_df['ID'] = y_df['Folder'].apply(extract_prefix)

y_df 加入分類結果並重新排序製成與 x_df 索引值相同的資料集

In [36]:
# sort y_df['ID']
y_df = y_df.sort_values(by='ID')
y_df = y_df.reset_index()

In [37]:
# 將 train_label 中的 'Classification' 列值賦值給 y_df 中的 'Classification' 列
Classification = []
i = 0
j = 0

numi = len(y_df)
numj = len(train_label)

while (i < numi)&(j < numj):
  if y_df['ID'][i] == train_label['ID'][j]:
    Classification.append(train_label['Classification'][j])
    i = i+1
  elif y_df['ID'][i] < train_label['ID'][j]:
    i = i+1
    Classification.append('NaN')
  else:
    j = j+1

# 顯示修改後的 DataFrame
y_df['Classification'] = Classification

In [38]:
# 將 y_df 回復順序與 x_df 一致
y_df = y_df.sort_values(by='index')
y_df = y_df.reset_index(drop=True)

# 刪除 index 行
y_df = y_df.drop(columns=['index'])
y_df

,Folder,ID,Classification
0,0010_loc-1_pine-side_01.jpg,10,1
1,0010_loc-1_pine-side_02.jpg,10,1
2,0011_loc-2_pine-side_02.jpg,11,3
3,0011_loc-2_pine-side_04.jpg,11,3
4,0014_loc-1_pine-side_03.jpg,14,3
...,...,...,...
81,0475_loc-2_pine-side_02.jpg,475,2
82,0475_loc-2_pine-side_03.jpg,475,2
83,0475_loc-2_pine-side_04.jpg,475,2
84,0478_loc-2_pine-side_03.jpg,478,2


將 y_df 儲存.csv檔

In [39]:
# 新增 x_df['Classification']
Classification = y_df['Classification']
print(Classification.shape)

# y_data.csv
y_df.to_csv('y_data.csv', index=False)

(86,)


##### 分類資料集

刪除 Classification = null 的列數

In [40]:
# 刪除 y_df['Classification'] = 'NaN'
y_df = y_df.dropna(subset=['Classification'])
print(y_df.shape)

(86, 3)


In [41]:
# 標籤編碼
num_class = 5 # 只有4種分類
y = np.array(y_df['Classification'])  # 1-4
y = keras.utils.to_categorical(y, num_classes=num_class)
y

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0.

#### train_x

x_padded

In [42]:
# 記錄每個數組的形狀
shapes = [array.shape for array in x]
print("x shape:", shapes)

# 將數據和標籤轉換為 NumPy 數組
# 找到最大形狀
max_shape = np.max([array.shape for array in x], axis=0)

# 使用 np.pad 將每個數組填充到相同的形狀
x_padded = [np.pad(array, [(0, max_shape[0] - array.shape[0]), (0, max_shape[1] - array.shape[1]), (0, max_shape[2] - array.shape[2]), (0, max_shape[3] - array.shape[3])], mode='constant') for array in x]

# 將填充後的列表轉換為 NumPy 數組
x_padded = np.array(x_padded)

# 打印填充後的 x 的形狀
print("Padded x shape:", x_padded.shape)

x shape: [(1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), 

轉為 DataFrame 並將分類結果=null的去掉

In [43]:
# 將四維陣列轉換為二維 DataFrame
x_reshaped = x_padded.reshape((x_padded.shape[0], -1))
x_df = pd.DataFrame(x_reshaped)

print("DataFrame 的形狀:", x_df.shape)

DataFrame 的形狀: (86, 230400)


In [44]:
x_df['Classification'] = Classification
print(x_df.shape)

# 刪除 y_df['Classification'] = 'NaN' 的列
x_df = x_df.dropna(subset=['Classification'])
print(x_df.shape)

(86, 230401)
(86, 230401)


##### 分類資料集

In [45]:
# 刪除 x_df['Classification']
x_df = x_df.drop(columns=['Classification'])
print(x_df.shape)

(86, 230400)


轉回 NumPy 陣列

In [46]:
# 將 DataFrame 的值轉換為 NumPy 陣列，並重新調整形狀
X_back = x_df.values.reshape((x_padded.shape[0], x_padded.shape[2], x_padded.shape[3]))

# 驗證恢復後的 NumPy 陣列的形狀
print("恢復後的 X 形狀:", X_back.shape)

# # 確認恢復後的數據是否與原數據一致
# assert np.array_equal(x_padded, X_back)

恢復後的 X 形狀: (86, 360, 640)


標準化 train_x

In [47]:
# 添加一個新的軸，使得形狀從 (num_samples, height, width) 變為 (num_samples, height, width, 1)
X = X_back[..., np.newaxis]

# 檢查新形狀
print("添加新軸後的 X 形狀:", X.shape)

# 標準化：將 X 的值縮放到 [0, 1] 範圍內
X_min = np.min(X)
X_max = np.max(X)

X = (X - X_min) / (X_max - X_min)

# 檢查標準化後的最大值和最小值
print("標準化後的 X 最大值:", np.max(X))
print("標準化後的 X 最小值:", np.min(X))

添加新軸後的 X 形狀: (86, 360, 640, 1)
標準化後的 X 最大值: 1.0
標準化後的 X 最小值: 0.0


### 訓練模型

In [48]:
# 劃分訓練集和測試集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [49]:
# 構建 CNN
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(X.shape[1], X.shape[2], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])


In [50]:
# 構建 CNN 模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_class, activation='softmax')
])

In [51]:
# 編譯模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
# 訓練模型
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/20
2/2 [==============================] - 20s 2s/step - loss: 7.1734 - accuracy: 0.0294 - val_loss: 14.7437 - val_accuracy: 0.2222
Epoch 2/20
2/2 [==============================] - 10s 1s/step - loss: 13.4112 - accuracy: 0.3382 - val_loss: 16.7923 - val_accuracy: 0.1111
Epoch 3/20
2/2 [==============================] - 10s 2s/step - loss: 13.7292 - accuracy: 0.2206 - val_loss: 2.9372 - val_accuracy: 0.6111
Epoch 4/20
2/2 [==============================] - 9s 1s/step - loss: 4.4561 - accuracy: 0.5000 - val_loss: 1.7929 - val_accuracy: 0.6111
Epoch 5/20
2/2 [==============================] - 10s 1s/step - loss: 2.8635 - accuracy: 0.5000 - val_loss: 1.4265 - val_accuracy: 0.3889
Epoch 6/20
2/2 [==============================] - 9s 1s/step - loss: 1.1835 - accuracy: 0.5147 - val_loss: 1.4697 - val_accuracy: 0.2778
Epoch 7/20
2/2 [==============================] - 9s 1s/step - loss: 0.9807 - accuracy: 0.5294 - val_loss: 0.9502 - val_accuracy: 0.6667
Epoch 8/20
2/2 [=================

### 評估模型

In [53]:
# 評估模型
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')
print(f'Test loss: {test_loss}')

1/1 [==============================] - 1s 588ms/step - loss: 1.1532 - accuracy: 0.7222
Test accuracy: 0.7222222089767456
Test loss: 1.1532349586486816


In [54]:
# 保存模型
model.save(f'image_model_{sideORbottom}_{version}.h5')

# Test data

In [55]:
mypath = 'K:\\DataMiningProject\\Test set\\'

x_test = []

count = 0
for (dirpath, dirnames, filenames) in os.walk(mypath):
  dirpath_list = dirpath.split('\\')

  # if count > 20:
  #   break


  if len(dirpath_list) > 5:
    for f in filenames:
      if ".jpg" in f:

        # if 'pine-bottom' in dirpath:
        if 'pine-side' in dirpath:
          # count += 1
          # if count > 20:
          #   break

          print(f, dirnames, dirpath_list[3], dirpath_list[5])
          image_path = f'{dirpath}/{f}'
          image = keras.utils.load_img(image_path, color_mode="grayscale", target_size=(360, 640))
          input_arr = np.array([keras.utils.img_to_array(image)])  # Convert single image to a batch.
          x_test.append(input_arr)

01.jpg [] 0005 pine-side
02.jpg [] 0005 pine-side
04.jpg [] 0005 pine-side
01.jpg [] 0008 pine-side
02.jpg [] 0008 pine-side
03.jpg [] 0008 pine-side
04.jpg [] 0008 pine-side
01.jpg [] 0008 pine-side
02.jpg [] 0008 pine-side
02.jpg [] 0013 pine-side
01.jpg [] 0018 pine-side
02.jpg [] 0018 pine-side
03.jpg [] 0018 pine-side
04.jpg [] 0018 pine-side
01.jpg [] 0025 pine-side
04.jpg [] 0025 pine-side
01.jpg [] 0026 pine-side
04.jpg [] 0026 pine-side
03.jpg [] 0028 pine-side
04.jpg [] 0028 pine-side
04.jpg [] 0042 pine-side
03.jpg [] 0046 pine-side
04.jpg [] 0054 pine-side
03.jpg [] 0061 pine-side
01.jpg [] 0069 pine-side
02.jpg [] 0069 pine-side
01.jpg [] 0073 pine-side
02.jpg [] 0073 pine-side
03.jpg [] 0073 pine-side
04.jpg [] 0073 pine-side
03.jpg [] 0094 pine-side
02.jpg [] 0108 pine-side
03.jpg [] 0108 pine-side
04.jpg [] 0108 pine-side
01.jpg [] 0116 pine-side
04.jpg [] 0116 pine-side
01.jpg [] 0117 pine-side
03.jpg [] 0117 pine-side
04.jpg [] 0117 pine-side
02.jpg [] 0147 pine-side


#### test_x

x_test_padded

In [56]:
# 記錄每個數組的形狀
shapes = [array.shape for array in x_test]
print("x shape:", shapes)

# 將數據和標籤轉換為 NumPy 數組
# 找到最大形狀
max_shape = np.max([array.shape for array in x_test], axis=0)

# 使用 np.pad 將每個數組填充到相同的形狀
x_test_padded = [np.pad(array, [(0, max_shape[0] - array.shape[0]), (0, max_shape[1] - array.shape[1]), (0, max_shape[2] - array.shape[2]), (0, max_shape[3] - array.shape[3])], mode='constant') for array in x_test]

# 將填充後的列表轉換為 NumPy 數組
x_test_padded = np.array(x_test_padded)

# 打印填充後的 x 的形狀
print("Padded x shape:", x_test_padded.shape)

x shape: [(1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), (1, 360, 640, 1), 

轉為 DataFrame 並將分類結果=null的去掉

In [57]:
# 將四維陣列轉換為二維 DataFrame
x_reshaped = x_test_padded.reshape((x_test_padded.shape[0], -1))
x_test_df = pd.DataFrame(x_reshaped)

print("DataFrame 的形狀:", x_test_df.shape)

DataFrame 的形狀: (103, 230400)


轉回 NumPy 陣列

In [58]:
# 將 DataFrame 的值轉換為 NumPy 陣列，並重新調整形狀
X_back = x_test_df.values.reshape((x_test_padded.shape[0], x_test_padded.shape[2], x_test_padded.shape[3]))

# 驗證恢復後的 NumPy 陣列的形狀
print("恢復後的 X 形狀:", X_back.shape)

# # 確認恢復後的數據是否與原數據一致
# assert np.array_equal(x_test_padded, X_back)

恢復後的 X 形狀: (103, 360, 640)


標準化 train_x

In [59]:
# 添加一個新的軸，使得形狀從 (num_samples, height, width) 變為 (num_samples, height, width, 1)
X_test = X_back[..., np.newaxis]

# 檢查新形狀
print("添加新軸後的 X_test 形狀:", X.shape)

# 標準化：將 X_test 的值縮放到 [0, 1] 範圍內
X_min = np.min(X_test)
X_max = np.max(X_test)

X_test = (X_test - X_min) / (X_max - X_min)

# 檢查標準化後的最大值和最小值
print("標準化後的 X_test 最大值:", np.max(X_test))
print("標準化後的 X_test 最小值:", np.min(X_test))

添加新軸後的 X_test 形狀: (86, 360, 640, 1)
標準化後的 X_test 最大值: 1.0
標準化後的 X_test 最小值: 0.0


In [60]:
pred_y = model.predict(X_test)

4/4 [==============================] - 3s 705ms/step


In [61]:
for i in range(len(pred_y)):
    print(i, np.argmax(pred_y[i]+1))

0 3
1 3
2 3
3 3
4 3
5 3
6 3
7 3
8 3
9 3
10 3
11 3
12 3
13 3
14 3
15 3
16 3
17 3
18 3
19 3
20 3
21 3
22 2
23 2
24 3
25 3
26 3
27 3
28 3
29 3
30 1
31 3
32 3
33 3
34 2
35 2
36 3
37 2
38 2
39 3
40 3
41 3
42 3
43 3
44 3
45 3
46 3
47 3
48 3
49 3
50 3
51 3
52 3
53 3
54 3
55 3
56 3
57 3
58 3
59 3
60 3
61 3
62 3
63 3
64 3
65 3
66 3
67 3
68 3
69 3
70 3
71 3
72 1
73 1
74 1
75 1
76 1
77 1
78 1
79 1
80 1
81 1
82 1
83 1
84 1
85 1
86 1
87 3
88 3
89 3
90 3
91 3
92 3
93 2
94 3
95 3
96 3
97 3
98 3
99 3
100 3
101 3
102 3
